# Homework 5

## FINM 35700 - Spring 2025

### UChicago Financial Mathematics

### Due Date: 2025-04-29

* Alex Popovici
* alex.popovici@uchicago.edu

This homework relies on following data files:

Government and corporate bonds
- the bond symbology file `bond_symbology`, 
- the "on-the-run" treasuries data file `govt_on_the_run`,
- the bond market data file `bond_market_prices_eod`,

Interest Rate & Credit Default Swaps
- the SOFR OIS symbology file `sofr_swap_symbology`,
- the SOFR swaps market data file `sofr_swaps_market_data_eod`,
- the CDS spreads market data file `cds_market_data_eod`.

In [1]:
# import tools from previous homeworks
from credit_market_tools import *

# Use static calculation/valuation date of 2024-12-13, matching data available in the market prices EOD file
calc_date = ql.Date(13, 12, 2024)
ql.Settings.instance().evaluationDate = calc_date

# Calculation/valuation date as pd datetime
as_of_date = pd.to_datetime('2024-12-13')
today = pd.to_datetime("2024-12-13") 
days_per_year = 365.25

In [2]:
# Load bond_symbology
bond_symbology = pd.read_excel("data/bond_symbology.xlsx")
bond_symbology['start_date'] = pd.to_datetime(bond_symbology['start_date'])
bond_symbology['cpn_first'] = pd.to_datetime(bond_symbology['cpn_first'])
bond_symbology['acc_first'] = pd.to_datetime(bond_symbology['acc_first'])
bond_symbology['maturity']   = pd.to_datetime(bond_symbology['maturity'])

bond_symbology['term'] = ((bond_symbology['maturity'] - bond_symbology['start_date']).dt.days / days_per_year)
bond_symbology['TTM'] = ((bond_symbology['maturity'] - today).dt.days / days_per_year)
bond_symbology['TTM'] = bond_symbology['TTM'].apply(lambda x: x if x > 0 else 0)
bond_symbology.head()

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,acc_first,maturity,mty_typ,rank,amt_out,country,currency,status,term,TTM
0,AAPL,Corp,BBG004HST0K7,US037833AL42,US912810UF39,AAPL 3.85 05/04/43,APPLE INC,GLOBAL,3.850,FIXED,...,2013-05-03,2043-05-04,AT MATURITY,Sr Unsecured,3000.0,US,USD,ACTV,30.001369,18.387406
1,AAPL,Corp,BBG006F8VWJ7,US037833AT77,US912810UF39,AAPL 4.45 05/06/44,APPLE INC,GLOBAL,4.450,FIXED,...,2014-05-06,2044-05-06,AT MATURITY,Sr Unsecured,1000.0,US,USD,ACTV,30.001369,19.394935
2,AAPL,Corp,BBG0081TNL50,US037833BA77,US912810UF39,AAPL 3.45 02/09/45,APPLE INC,GLOBAL,3.450,FIXED,...,2015-02-09,2045-02-09,AT MATURITY,Sr Unsecured,2000.0,US,USD,ACTV,30.001369,20.158795
3,AAPL,Corp,BBG008N1BQC1,US037833BH21,US912810UF39,AAPL 4 3/8 05/13/45,APPLE INC,GLOBAL,4.375,FIXED,...,2015-05-13,2045-05-13,AT MATURITY,Sr Unsecured,2000.0,US,USD,ACTV,30.001369,20.413415
4,AAPL,Corp,BBG00C7QB7Q2,US037833BY53,US91282CLY56,AAPL 3 1/4 02/23/26,APPLE INC,GLOBAL,3.250,FIXED,...,2016-02-23,2026-02-23,CALLABLE,Sr Unsecured,3250.0,US,USD,ACTV,10.001369,1.196441


In [3]:
# Load bond_market_prices_eod
bond_market_prices_eod = pd.read_excel("data/bond_market_prices_eod.xlsx")
bond_market_prices_eod["date"] = pd.to_datetime(bond_market_prices_eod["date"])
display(bond_market_prices_eod.head())

,date,class,ticker,isin,figi,bidPrice,askPrice,accrued,bidYield,askYield
0,2024-12-13,Corp,AAPL,US037833BX70,BBG00C7QBG91,93.228,93.809,1.4595,5.180,5.132
1,2024-12-13,Corp,AAPL,US037833EK23,BBG011ZS1X57,63.723,64.232,0.9835,5.242,5.194
2,2024-12-13,Corp,AAPL,US037833DW79,BBG00TN2PN26,63.716,64.215,0.2585,5.253,5.205
3,2024-12-13,Corp,AAPL,US037833EF38,BBG00Z3VQ626,63.262,63.766,0.9420,5.249,5.200
4,2024-12-13,Corp,AAPL,US037833CD08,BBG00DHQX9M5,82.278,82.848,1.4120,5.227,5.177


In [4]:
# Load govt_on_the_run
govt_on_the_run = pd.read_excel("data/govt_on_the_run.xlsx")
on_the_run_list = ["2Y","3Y","5Y","7Y","10Y","20Y","30Y"]
pattern = r"^GT(\d+)([A-Z])?\sGovt$"
govt_on_the_run[["tenor", "suffix"]] = govt_on_the_run["ticker"].str.extract(pattern)
govt_on_the_run["tenor"] = govt_on_the_run["tenor"] + "Y"
display(govt_on_the_run.head())

,ticker,date,figi,isin,tenor,suffix
0,GT10 Govt,2024-12-13,BBG01QKHSMP5,US91282CLW90,10Y,NaN
1,GT10B Govt,2024-12-13,BBG01P1YBJQ5,US91282CLF67,10Y,B
2,GT10C Govt,2024-12-13,BBG01MPC8VJ9,US91282CKQ32,10Y,C
3,GT2 Govt,2024-12-13,BBG01QZFYJV6,US91282CLY56,2Y,NaN
4,GT20 Govt,2024-12-13,BBG01QVTC1Y0,US912810UF39,20Y,NaN


In [5]:
# Load sofr_swaps_symbology
sofr_sym = pd.read_excel('data/sofr_swaps_symbology.xlsx')
sofr_sym.head()

,figi,ticker,class,bbg,name,tenor,type,dcc,exchange,country,currency,status
0,BBG00KFWPJJ9,USOSFR1,Curncy,USOSFR1 Curncy,USD OIS ANN VS SOFR 1Y,1,SWAP,ACT/360,NONE,US,USD,ACTV
1,BBG00KFWPJX3,USOSFR2,Curncy,USOSFR2 Curncy,USD OIS ANN VS SOFR 2Y,2,SWAP,ACT/360,NONE,US,USD,ACTV
2,BBG00KFWPK15,USOSFR3,Curncy,USOSFR3 Curncy,USD OIS ANN VS SOFR 3Y,3,SWAP,ACT/360,NONE,US,USD,ACTV
3,BBG00KFWPK51,USOSFR5,Curncy,USOSFR5 Curncy,USD OIS ANN VS SOFR 5Y,5,SWAP,ACT/360,NONE,US,USD,ACTV
4,BBG00KFWPK79,USOSFR7,Curncy,USOSFR7 Curncy,USD OIS ANN VS SOFR 7Y,7,SWAP,ACT/360,NONE,US,USD,ACTV


In [6]:
# Load sofr_swaps_market_data_eod
sofr_mkt = pd.read_excel('data/sofr_swaps_market_data_eod.xlsx')
sofr_mkt['date'] = pd.to_datetime(sofr_mkt['date'])
sofr_mkt.head()

,date,figi,bidRate,askRate,midRate
0,2024-01-02,BBG00KFWPJJ9,4.7960,4.8046,4.80030
1,2024-01-02,BBG00KFWPJX3,4.1368,4.1452,4.14100
2,2024-01-02,BBG00KFWPK15,3.8258,3.8327,3.82925
3,2024-01-02,BBG00KFWPK51,3.5907,3.5943,3.59250
4,2024-01-02,BBG00KFWPK79,3.5297,3.5333,3.53150


In [7]:
# Load cds_market_data_eod
cds = pd.read_excel('data/cds_market_data_eod.xlsx')
cds['date'] = pd.to_datetime(cds['date'])
cds.sort_values(by='date', inplace=True)
display(cds.head())

,date,ticker,short_name,tier,sector,region,currency,doc_clause,running_coupon,cds_assumed_recovery,par_spread_1y,par_spread_2y,par_spread_3y,par_spread_5y,par_spread_7y,par_spread_10y
0,2024-01-02,IBM,Intl Business Machs Corp,SNRFOR,Technology,N.Amer,USD,XR14,0.01,0.4,13.6831,18.8194,28.3917,44.7053,62.1494,69.1972
1,2024-01-03,IBM,Intl Business Machs Corp,SNRFOR,Technology,N.Amer,USD,XR14,0.01,0.4,14.2256,19.6610,29.4493,46.4866,63.6475,71.4311
2,2024-01-04,IBM,Intl Business Machs Corp,SNRFOR,Technology,N.Amer,USD,XR14,0.01,0.4,13.8318,19.1828,28.8454,45.4735,62.6543,70.9180
3,2024-01-05,IBM,Intl Business Machs Corp,SNRFOR,Technology,N.Amer,USD,XR14,0.01,0.4,13.6181,18.7703,28.3417,44.7575,61.9778,70.2746
4,2024-01-08,IBM,Intl Business Machs Corp,SNRFOR,Technology,N.Amer,USD,XR14,0.01,0.4,13.4433,18.3692,27.7599,43.8548,60.8378,68.8914


-----------------------------------------------------------
# Problem 1: Credit Default Swaps (hazard rate model)

## When computing sensitivities, assume "everything else being equal" (ceteris paribus).

For a better understanding of dependencies, you can use the CDS valuation formulas in the simple hazard rate model (formulas[45] and [46] in Lecture 4).

\begin{align}
PV_{CDS\_PL}\left(c,r,h,R,T\right) = \frac{c}{4 \cdot \left(e^{\left(r+h\right)/4}-1 \right)} \cdot\left[1-e^{-T\cdot\left(r+h\right)}\right] \simeq \frac{c}{r+h} \cdot\left[1-e^{-T\cdot\left(r+h\right)}\right]
\end{align}

\begin{align}
PV_{CDS\_DL}\left(c,r,h,R,T\right) = \frac{\left(1-R\right)\cdot h}{r+h} \cdot\left[1-e^{-T\cdot\left(r+h\right)}\right]
\end{align}

\begin{align}
PV_{CDS} = PV_{CDS\_PL} - PV_{CDS\_DL} \simeq \frac{c - \left(1-R\right)\cdot h}{r+h} \cdot\left[1-e^{-T\cdot\left(r+h\right)}\right]
\end{align}

\begin{align}
CDS\_ParSpread = c \cdot \frac{PV_{CDS\_DL}}{PV_{CDS\_PL}} \simeq \left(1-R\right)\cdot h
\end{align}


## a. True or False (CDS Premium Leg PV)

1. CDS premium leg PV is increasing in CDS Par Spread **TRUE**
2. CDS premium leg PV is increasing in interest rate  **FALSE**
2. CDS premium leg PV is increasing in hazard rate  **FALSE**
3. CDS premium leg PV is increasing in recovery rate  **FALSE**
4. CDS premium leg PV is increasing in coupon **TRUE**
5. CDS premium leg PV is increasing in CDS maturity  **TRUE**


## b. True or False (CDS Default Leg PV)

1. CDS default leg PV is increasing in CDS Par Spread  **FALSE**
2. CDS default leg PV is increasing in interest rate  **FALSE**
3. CDS default leg PV is increasing in hazard rate  **TRUE**
4. CDS default leg PV is increasing in recovery rate  **FALSE**
5. CDS default leg PV is increasing in coupon  **FALSE**
6. CDS default leg PV is increasing in CDS maturity  **TRUE**

## c. True or False (CDS PV)


1. CDS PV is increasing in CDS Par Spread  **TRUE**
2. CDS PV is increasing in interest rate  **FALSE**
3. CDS PV is increasing in hazard rate  **FALSE**
4. CDS PV is increasing in recovery rate  **TRUE**
5. CDS PV is increasing in coupon  **TRUE**
6. CDS PV is increasing in CDS maturity  **TRUE**

## d. True or False (CDS Par Spread)


1. CDS Par Spread is increasing in interest rates  **FALSE**
2. CDS Par Spread is increasing in hazard rate  **TRUE**
3. CDS Par Spread is increasing in recovery rate  **FALSE**
4. CDS Par Spread is increasing in coupon  **FALSE**
5. CDS Par Spread is increasing in CDS maturity  **FALSE**

-----------------------------------------------------------
# Problem 2: Perpetual CDS
We are interested in a perpetual CDS contract (infinite maturity) on a face notional of $100, flat interest rate of 4% and coupon of 1% (quarterly payments).

For simplicity, we assuming a flat hazard rate of 2% per annum, a recovery rate of 40%, T+0 settlement and zero accrued.

Use the simple CDS valuation formulas derived in Session 4 as a template.

**Assumptions**  
- Notional \(N = 100\)  
- Flat risk-free rate \(r = 4\%\) p.a.  
- CDS coupon \(c = 1\%\) p.a., paid quarterly (\(\Delta t = 0.25\) yr)  
- Flat hazard rate \(h = 2\%\) p.a.  
- Recovery \(R = 40\%\)  
- T+0 settlement, zero accrued  

## a. Compute the fair value of the CDS premium and default legs.


**Premium Leg (discrete quarterly coupons)**  
$
\mathrm{PV}_{\rm PL}
= N\,c\,\Delta t\sum_{k=1}^\infty e^{-(r+h)\,k\Delta t}
= N\,c\,\Delta t\,\frac{e^{-(r+h)\,\Delta t}}{1 - e^{-(r+h)\,\Delta t}}.
$

**Default Leg**  
$
\mathrm{PV}_{\rm DL}
= N\,(1-R)\int_{0}^{\infty}h\,e^{-(r+h)s}\,ds
= N\,(1-R)\,\frac{h}{r+h}.
$


In [8]:
# Given data
N  = 100.0      # Notional
r  = 0.04       # Risk-free rate (annual)
c  = 0.01       # Coupon rate (annual)
dt = 0.25       # Quarterly payment interval
h  = 0.02       # Hazard rate (annual)
R  = 0.40       # Recovery rate

In [9]:
disc = np.exp(-(r + h) * dt)

In [10]:
# Premium leg PV 
pv_pl = N * c * dt * disc / (1 - disc)
# Default leg PV
pv_dl = N * (1 - R) * h / (r + h)

print(f"PV CDS Premuim Leg: {pv_pl}")
print(f"PV CDS Default Leg: {pv_dl}")

PV CDS Premuim Leg: 16.541979165494784
PV CDS Default Leg: 20.0


## b. Compute the CDS PV, the CDS Upfront and the CDS Par Spread.

1. **CDS PV**  
   $
   \boxed{
     \mathrm{PV_{CDS}} = \mathrm{PV_{PL}} - \mathrm{PV_{DL}}
   }
   $
2. **Upfront**  
   $
   \boxed{
     \text{Upfront} = -\,\mathrm{PV_{CDS}}
   }
   $
3. **Par Spread** \(s\)  
   $
     \mathrm{PV_{DL}}
     = s \times \underbrace{
         \sum_{k=1}^\infty \Delta t\,e^{-(r+h)\,k\Delta t}
       }_{\displaystyle\mathrm{Duration}}
     \;\;\Longrightarrow\;\;
     \boxed{
       s = \frac{\mathrm{PV_{DL}}}{\mathrm{Duration}}
     }.
   $
   
   $
     \mathrm{Duration}
     = \sum_{k=1}^\infty\Delta t\,e^{-(r+h)\,k\Delta t}
     = \Delta t\,\frac{e^{-(r+h)\,\Delta t}}
                    {1 - e^{-(r+h)\,\Delta t}}.
   $

In [11]:
# 1. Net PV of the CDS
pv_cds = pv_pl - pv_dl
# 2. Upfront fee 
upfront = -pv_cds
# 3. Par spread
duration = dt * disc / (1 - disc)    
s = pv_dl / (N * duration)
# Par spread in basis points
s_bps = s * 1e4


print(f"PV CDS PV: {pv_cds}")
print(f"PV CDS Upfront: {upfront}")
print(f"PV CDS Par Spread (bps): {s_bps}")

PV CDS PV: -3.4580208345052164
PV CDS Upfront: 3.4580208345052164
PV CDS Par Spread (bps): 120.90451692575193


## c. Compute the following CDS risk sensitivities:
- IR01 (PV sensitivity to Interest Rate change of '-1bp')
- HR01 (PV sensitivity to Hazard Rate change of '-1bp')
- REC01 (PV sensitivity to Recovery Rate change of '+1%')

using the scenario method.


$
\begin{aligned}
\text{IR01} &=
\mathrm{PV_{CDS}}(r - 0.0001,\;h,\;R)
\;-\;
\mathrm{PV_{CDS}}(r,\;h,\;R),\\
\text{HR01} &=
\mathrm{PV_{CDS}}(r,\;h - 0.0001,\;R)
\;-\;
\mathrm{PV_{CDS}}(r,\;h,\;R),\\
\text{REC01} &=
\mathrm{PV_{CDS}}(r,\;h,\;R + 0.01)
\;-\;
\mathrm{PV_{CDS}}(r,\;h,\;R).
\end{aligned}
$

In [12]:
def pv_cds_model(r_, h_, R_):
    disc_ = np.exp(-(r_ + h_) * dt)
    pv_pl_ = N * c * dt * disc_ / (1 - disc_)
    pv_dl_ = N * (1 - R_) * h_ / (r_ + h_)
    return pv_pl_ - pv_dl_

base_pv = pv_cds

ir01 = pv_cds_model(r + 1e-4, h, R) - base_pv # IR01: +1 bp move in r
hr01 = pv_cds_model(r, h + 1e-4, R) - base_pv # HR01: +1 bp move in h
rec01 = pv_cds_model(r, h, R + 0.01) - base_pv # REC01: +1% move in R

print(f"IR01  = {ir01:.6f}")
print(f"HR01  = {hr01:.6f}")
print(f"REC01 = {rec01:.6f}")

IR01  = 0.005547
HR01  = -0.094287
REC01 = 0.333333


## d. At what time T does the (implied) default probability over next 10 years (from $[T, T+10]$) drop to 10%?

\begin{align}
\mathbb{P} \left(\tau \in [T, T+10] \right) = 10/100
\end{align}


1. **Survival probability** to time \(t\)  
   $
   SP(0,t) \;=\; P(\tau > t)
   = e^{-\,h\,t}
   \quad\text{(since }h\text{ constant).}
   $

2. **Probability of default** in \([T,\,T+10]\)  
   $
   P\bigl(T<\tau\le T+10\bigr)
   = SP(0,T)\;-\;SP(0,T+10)
   = e^{-\,h\,T} \;-\; e^{-\,h\,(T+10)}.
   $

   $
   e^{-hT} \;-\; e^{-h(T+10)} = 0.10
   \quad\Longrightarrow\quad
   e^{-hT}\bigl(1 - e^{-10\,h}\bigr) = 0.10
   \quad\Longrightarrow\quad
   e^{-hT} = \frac{0.10}{1 - e^{-10\,h}},
   $
   $
   \boxed{
     T = -\frac{1}{h}\,\ln\!\Bigl[\tfrac{0.10}{1 - e^{-10\,h}}\Bigr].
   }
   $

In [13]:
numer = 0.10
denom = 1 - np.exp(-10 * h)
T = - (1.0 / h) * np.log(numer / denom)
T

29.74066460117629

-----------------------------------------------------------
# Problem 3: Pricing risky bonds in the hazard rate model
## This is building upon
- Homework 2 "Problem 2: US Treasury yield curve calibration (On-The-Runs)",
- Homework 4 "Problem 2: US SOFR swap curve calibration" and
- Homework 4 "Problem 3: CDS Hazard Rate calibration".

## a. Prepare the market data
### Load the symbology + market data dataframes. Calibrate the following curves as of 2024-12-13:
- the "on-the-run" US Treasury curve,
- the US SOFR curve and 
- the IBM CDS hazard rate curve (on the top of SOFR discount curve).


In [14]:
US_treasury_bonds = bond_symbology[(bond_symbology["class"] == "Govt") & (bond_symbology["ticker"] == "T")]
on_the_run_bonds = govt_on_the_run[(govt_on_the_run["tenor"].isin(on_the_run_list)) & (govt_on_the_run["suffix"].isna())].copy()
us_treasuries_on_the_run = pd.merge(US_treasury_bonds, on_the_run_bonds, on=["figi", "isin"]).drop(["suffix", "ticker_y", "date", "tenor"], axis=1).rename(columns={"ticker_x": "ticker"})
bond_market_prices_eod = bond_market_prices_eod[bond_market_prices_eod["date"]=="2024-12-13"]

merged_treasuries = pd.merge(US_treasury_bonds,
    bond_market_prices_eod,
    on=["ticker","isin","figi"],
    how="inner"
)
merged_treasuries = merged_treasuries.drop(columns=["class_y"])
merged_treasuries.rename(columns={"class_x": "class"}, inplace=True)
merged_treasuries["midPrice"] = (merged_treasuries["bidPrice"] + merged_treasuries["askPrice"])/2
merged_treasuries["midYield"] = (merged_treasuries["bidYield"] + merged_treasuries["askYield"])/2

merge_columns = list(set(merged_treasuries.columns).intersection(set(us_treasuries_on_the_run)))
merged_us_treasuries_on_the_run = pd.merge(merged_treasuries, us_treasuries_on_the_run, on=merge_columns, how="inner")
merged_us_treasuries_on_the_run

,ticker,class,figi,isin,und_bench_isin,security,name,type,coupon,cpn_type,...,term,TTM,date,bidPrice,askPrice,accrued,bidYield,askYield,midPrice,midYield
0,T,Govt,BBG01QKHSMP5,US91282CLW90,US91282CLW90,T 4 1/4 11/15/34,US TREASURY N/B,US GOVERNMENT,4.250,FIXED,...,9.998631,9.921971,2024-12-13,98.8125,98.8281,0.36330,4.399,4.397,98.82030,4.3980
1,T,Govt,BBG01QKHSL31,US912810UE63,US912810UE63,T 4 1/2 11/15/54,US TREASURY N/B,US GOVERNMENT,4.500,FIXED,...,29.998631,29.921971,2024-12-13,98.3281,98.3594,0.38475,4.603,4.601,98.34375,4.6020
2,T,Govt,BBG01QZFYJV6,US91282CLY56,US91282CLY56,T 4 1/4 11/30/26,US TREASURY N/B,US GOVERNMENT,4.250,FIXED,...,1.993155,1.963039,2024-12-13,100.0000,100.0078,0.18750,4.249,4.245,100.00390,4.2470
3,T,Govt,BBG01QZFYD58,US91282CMA61,US91282CMA61,T 4 1/8 11/30/29,US TREASURY N/B,US GOVERNMENT,4.125,FIXED,...,4.993840,4.963723,2024-12-13,99.4375,99.4453,0.18160,4.252,4.250,99.44140,4.2510
4,T,Govt,BBG01QZFYCF9,US91282CLZ22,US91282CLZ22,T 4 1/8 11/30/31,US TREASURY N/B,US GOVERNMENT,4.125,FIXED,...,6.992471,6.962355,2024-12-13,98.7969,98.8125,0.18160,4.327,4.324,98.80470,4.3255
5,T,Govt,BBG01QVTC1Y0,US912810UF39,US912810UF39,T 4 5/8 11/15/44,US TREASURY N/B,US GOVERNMENT,4.625,FIXED,...,19.953457,19.923340,2024-12-13,99.2500,99.2813,0.39645,4.683,4.681,99.26565,4.6820
6,T,Govt,BBG01R4Z7Y32,US91282CMB45,US91282CMB45,T 4 12/15/27,US TREASURY N/B,US GOVERNMENT,4.000,FIXED,...,2.995209,3.003422,2024-12-13,99.3750,99.3828,0.01170,4.224,4.221,99.37890,4.2225


In [15]:
sofr_mkt[sofr_mkt["date"]=="2024-12-13"]
df_sofr_asof = sofr_sym.merge(sofr_mkt, on='figi').sort_values('date')
df_sofr_asof

,figi,ticker,class,bbg,name,tenor,type,dcc,exchange,country,currency,status,date,bidRate,askRate,midRate
0,BBG00KFWPJJ9,USOSFR1,Curncy,USOSFR1 Curncy,USD OIS ANN VS SOFR 1Y,1,SWAP,ACT/360,NONE,US,USD,ACTV,2024-01-02,4.7960,4.8046,4.80030
470,BBG00KFWPK15,USOSFR3,Curncy,USOSFR3 Curncy,USD OIS ANN VS SOFR 3Y,3,SWAP,ACT/360,NONE,US,USD,ACTV,2024-01-02,3.8258,3.8327,3.82925
1175,BBG00KFWPKB4,USOSFR10,Curncy,USOSFR10 Curncy,USD OIS ANN VS SOFR 10Y,10,SWAP,ACT/360,NONE,US,USD,ACTV,2024-01-02,3.5142,3.5179,3.51605
1410,BBG00KFWPKF0,USOSFR20,Curncy,USOSFR20 Curncy,USD OIS ANN VS SOFR 20Y,20,SWAP,ACT/360,NONE,US,USD,ACTV,2024-01-02,3.5179,3.5230,3.52045
1644,BBG00KFWPKH8,USOSFR30,Curncy,USOSFR30 Curncy,USD OIS ANN VS SOFR 30Y,30,SWAP,ACT/360,NONE,US,USD,ACTV,2024-01-02,3.3466,3.3522,3.34940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,BBG00KFWPK15,USOSFR3,Curncy,USOSFR3 Curncy,USD OIS ANN VS SOFR 3Y,3,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,3.9883,3.9944,3.99135
469,BBG00KFWPJX3,USOSFR2,Curncy,USOSFR2 Curncy,USD OIS ANN VS SOFR 2Y,2,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,4.0524,4.0585,4.05545
234,BBG00KFWPJJ9,USOSFR1,Curncy,USOSFR1 Curncy,USD OIS ANN VS SOFR 1Y,1,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,4.1858,4.1958,4.19080
1643,BBG00KFWPKF0,USOSFR20,Curncy,USOSFR20 Curncy,USD OIS ANN VS SOFR 20Y,20,SWAP,ACT/360,NONE,US,USD,ACTV,2024-12-13,3.9246,3.9304,3.92750


In [16]:
cds_ibm = cds[(cds['date'] == '2024-12-13') & (cds['ticker'] == 'IBM')]
cols = ['par_spread_1y', 'par_spread_2y', 'par_spread_3y', 'par_spread_5y', 'par_spread_7y', 'par_spread_10y']
row = cds_ibm.iloc[0]
cds_par_spreads_bps = [row[c] for c in cols]

In [17]:
# tsy_yield_curve calibration
govt_combined_otr = merged_us_treasuries_on_the_run    # TODO: Follow Homework 2 Problem 2 and populate the US Treasury On-The-Run symbology + market data frame !!!
tsy_yield_curve = calibrate_yield_curve_from_frame(calc_date, govt_combined_otr, 'midPrice')
tsy_yield_curve_handle = ql.YieldTermStructureHandle(tsy_yield_curve)

# sofr_yield_curve calibration
sofr_combined = df_sofr_asof    # TODO: Follow Homework 3 Problem 3 and populate the SOFR symbology + market data frame !!!
sofr_yield_curve = calibrate_sofr_curve_from_frame(calc_date, sofr_combined, 'midRate')
sofr_yield_curve_handle = ql.YieldTermStructureHandle(sofr_yield_curve)


# hazard_rate_curve calibrated to IBM CDS par spreads
hazard_rate_curve = calibrate_cds_hazard_rate_curve(
    calc_date,
    sofr_yield_curve_handle,
    cds_par_spreads_bps,
    cds_recovery_rate=0.40
)    # TODO: Follow Homework 3 Problem 4 and create the IBM hazard rate curve !!!
default_prob_curve_handle = ql.DefaultProbabilityTermStructureHandle(hazard_rate_curve)

## b. Create the IBM risky bond objects
### Identify the following 3 IBM fixed rate bonds in the symbology table and create the corresponding fixed rate bonds (3 bond objects).

- security = 'IBM 3.3 01/27/27' / figi = 'BBG00FVNGFP3'
- security = 'IBM 6 1/2 01/15/28' / figi = 'BBG000058NM4'
- security = 'IBM 3 1/2 05/15/29' / figi = 'BBG00P3BLH14'


Use the create_bond_from_symbology() function (discussed in from Homework 2) to create the bonds objects.

Display the bond cashflows using the get_bond_cashflows() function.

In [18]:
ibm_figis = [
    'BBG00FVNGFP3',   # IBM 3.3 01/27/27
    'BBG000058NM4',   # IBM 6 1/2 01/15/28
    'BBG00P3BLH14'    # IBM 3 1/2 05/15/29
]


# build and display schedules
for figi in ibm_figis:
    details = bond_symbology[bond_symbology['figi']==figi].iloc[0].to_dict()
    bond = create_bond_from_symbology(details)
    cf_df = get_bond_cashflows(bond, calc_date)
    print(f"Cash-flows for {details['security']} (FIGI={figi}):")
    display(cf_df)
    print("\n")

Cash-flows for IBM 3.3 01/27/27 (FIGI=BBG00FVNGFP3):


,CashFlowDate,CashFlowYearFrac,CashFlowAmount
15,"January 27th, 2025",0.122222,1.65
16,"July 27th, 2025",0.622222,1.65
17,"January 27th, 2026",1.122222,1.65
18,"July 27th, 2026",1.622222,1.65
19,"January 27th, 2027",2.122222,1.65
20,"January 27th, 2027",2.122222,100.00




Cash-flows for IBM 6 1/2 01/15/28 (FIGI=BBG000058NM4):


,CashFlowDate,CashFlowYearFrac,CashFlowAmount
54,"January 15th, 2025",0.088889,3.25
55,"July 15th, 2025",0.588889,3.25
56,"January 15th, 2026",1.088889,3.25
57,"July 15th, 2026",1.588889,3.25
58,"January 15th, 2027",2.088889,3.25
59,"July 15th, 2027",2.588889,3.25
60,"January 15th, 2028",3.088889,3.25
61,"January 15th, 2028",3.088889,100.00




Cash-flows for IBM 3 1/2 05/15/29 (FIGI=BBG00P3BLH14):


,CashFlowDate,CashFlowYearFrac,CashFlowAmount
11,"May 15th, 2025",0.422222,1.75
12,"November 15th, 2025",0.922222,1.75
13,"May 15th, 2026",1.422222,1.75
14,"November 15th, 2026",1.922222,1.75
15,"May 15th, 2027",2.422222,1.75
16,"November 15th, 2027",2.922222,1.75
17,"May 15th, 2028",3.422222,1.75
18,"November 15th, 2028",3.922222,1.75
19,"May 15th, 2029",4.422222,1.75
20,"May 15th, 2029",4.422222,100.00


## c. Compute CDS-implied (intrinsic) prices for the IBM fixd rate bonds

Price the 3 IBM bonds using the CDS-calibrated hazard rate curve for IBM (via RiskyBondEngine, discussed in the QuantLib Advanced examples notebook).

Display the clean prices and yields for the 3 test bonds.

You can use the example code below.


In [33]:
details = bond_symbology[bond_symbology["figi"]==ibm_figis[2]].iloc[0].to_dict()
details

{'ticker': 'IBM',
 'class': 'Corp',
 'figi': 'BBG00P3BLH14',
 'isin': 'US459200KA85',
 'und_bench_isin': 'US91282CMA61',
 'security': 'IBM 3 1/2 05/15/29',
 'name': 'IBM CORP',
 'type': 'GLOBAL',
 'coupon': 3.5,
 'cpn_type': 'FIXED',
 'dcc': '30/360',
 'cpn_freq': 2,
 'days_settle': 1,
 'start_date': Timestamp('2019-05-15 00:00:00'),
 'cpn_first': Timestamp('2019-11-15 00:00:00'),
 'acc_first': Timestamp('2019-05-15 00:00:00'),
 'maturity': Timestamp('2029-05-15 00:00:00'),
 'mty_typ': 'AT MATURITY',
 'rank': 'Sr Unsecured',
 'amt_out': 3250.0,
 'country': 'US',
 'currency': 'USD',
 'status': 'ACTV',
 'term': 10.001368925393566,
 'TTM': 4.418891170431212}

In [34]:
# flat_recovery_rate: use market convention of 40% for "Senior Unsecured" Debt
flat_recovery_rate = 0.40

fixed_rate_bond = create_bond_from_symbology(details)    # TODO: Pick one of the 3 IBM test bonds !!!

# Risky bond engine uses the calibrated CDS hazard rate curve for pricing credit default risk 
risky_bond_engine = ql.RiskyBondEngine(default_prob_curve_handle, flat_recovery_rate, sofr_yield_curve_handle)

fixed_rate_bond.setPricingEngine(risky_bond_engine)

corpBondModelPrice = fixed_rate_bond.cleanPrice()

corpBondModelYield = fixed_rate_bond.bondYield(corpBondModelPrice, ql.Thirty360(ql.Thirty360.USA), ql.Compounded, ql.Semiannual) * 100

print('corpBondCleanPrice', corpBondModelPrice)
print('corpBondYield', corpBondModelYield)

RuntimeError: 1st iteration: failed at 1st alive instrument, pillar December 22nd, 2025, maturity December 22nd, 2025, reference date December 13th, 2024: more than one instrument with pillar December 16th, 2025

## d. Compute the "intrinsic" vs market price basis for the IBM bonds

Load the market mid prices and yields from the bond market data dataframe as of 2024-12-13. 

Compute and display the basis between the "CDS-implied intrinsic" vs market prices and yields:

- basisPrice = modelPrice - midPrice
- basisYield = modelYield - midYield


Are the CDS intrinsic prices lower or higher than the bond prices observed on the market? What factors could explain the basis?

-----------------------------------------------------------
# Problem 4: Compute scenario sensitivities for risky bonds
## a. Compute scenario IR01s and Durations for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 1. 

Compute the scenario IR01 and Durations using a '-1bp' interest rate shock, as described in Section 6. "Market Data Scenarios" in the QuantLib Basics notebook.

Display the computed scenario IR01 and Durations.

Remember that IR01 = Dirty_Price * Duration.


## b. Compute analytical IR01s and Durations for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 1. 

Compute and display the analytical IR01 and Durations 

Compare the analytic IR01s vs. the scenario IR01s. Are they expected to be similar?

## c. Compute scenario CS01s (credit spread sensitivities) for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 3. 

Apply a '-1bp' (parallel shift) scenario to the IBM CDS Par Spread quotes and re-calibrate the scenario hazard rate curve. 

Create a new scenario RiskyBondEngine, using the scenario hazard rate curve.

Reprice the risky bonds on the scenario RiskyBondEngine (using the bumped hazard rate curve) to obtain the '-1bp' scenario CS01 (credit spread sensitivities).

Compare the scenario CS01s vs analytic IR01s. Are they expected to be similar?

## d. Compute scenario REC01 (recovery rate sensitivity) for the 3 IBM bonds
Use the 3 IBM test bonds defined in Problem 1. 

Apply a +1% scenario bump to the IBM recovery rate (bump the flat_recovery_rate parameter by 1%, from 40% to 41%).

Create a new scenario RiskyBondEngine, using the scenario new recovery rate.

Reprice the risky bonds on the scenario RiskyBondEngine (using the bumped recovery rate) to obtain the +1% scenario REC01 (recovery rate sensitivity).
